<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/OAI_fine_tuning_customer_support_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers huggingface

In [ ]:
!pip install openai==1.55.3 httpx==0.27.2


In [ ]:
!pip install httpx==0.27.2

In [ ]:
!pip install datasets

In [ ]:
# Import necessary libraries
import os
import pandas as pd
import json
from openai import OpenAI

In [ ]:
from datasets import load_dataset

ds = load_dataset("Kaludi/Customer-Support-Responses")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['query', 'response'],
        num_rows: 74
    })
})

In [ ]:
df = pd.DataFrame(ds["train"])
df

,query,response
0,My order hasn't arrived yet.,We apologize for the inconvenience. Can you pl...
1,I received a damaged product.,We apologize for the inconvenience. Can you pl...
2,I need to return an item.,Certainly. Please provide your order number an...
3,I want to change my shipping address.,No problem. Can you please provide your order ...
4,I have a question about my bill.,We'd be happy to help. Can you please provide ...
...,...,...
69,How do I schedule a consultation or appointment?,We'd be happy to help. Can you please provide ...
70,Can I get a copy of my receipt?,Certainly. Can you please provide your order n...
71,Can I use a competitor's coupon at your store?,"In some cases, we may accept competitor coupon..."
72,Do you have a recycling program?,"Yes, we do have a recycling program. Can you p..."


In [ ]:
with open("data.jsonl", "w") as f:
  for _, row in df.iterrows():
    json_line = {
        "messages": [
            {"role":"system", "content":"you are a helpful customer support assistant"},
            {"role": "user", "content": row["query"]},
            {"role": "assistant", "content": row["response"]}
        ]
        }
    f.write(json.dumps(json_line) + "\n")

In [ ]:
os.environ["OPENAI_API_KEY"] = "openai_key"

In [ ]:
client = OpenAI()

In [ ]:
# create a file
client.files.create(
    file=open("data.jsonl", "rb"),
    purpose="fine-tune"
)

FileObject(id='file-H4CJZt8fPUph6ohyqsrAr1', bytes=24432, created_at=1734408046, filename='data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
# fine-tuning a job
client.fine_tuning.jobs.create(
    training_file="file-H4CJZt8fPUph6ohyqsrAr1",
    model="gpt-3.5-turbo-0125"
)

FineTuningJob(id='ftjob-3JBoNz2DNqXiaDBfSECqiyEN', created_at=1734408231, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-bQVIhPzVjnoakAjRyRFBoo5Y', result_files=[], seed=362833245, status='validating_files', trained_tokens=None, training_file='file-H4CJZt8fPUph6ohyqsrAr1', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:

# fine_tuning.jobs.list
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-3JBoNz2DNqXiaDBfSECqiyEN', created_at=1734408231, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-bQVIhPzVjnoakAjRyRFBoo5Y', result_files=[], seed=362833245, status='running', trained_tokens=None, training_file='file-H4CJZt8fPUph6ohyqsrAr1', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None), FineTuningJob(id='ftjob-47oADAFHIraKWFj3MIVzC29w', created_at=1733667943, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::AcCbn4dP', finished_at=1733668293, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-bQVIhPzVjnoakAjRyRFBoo5Y', resul

In [ ]:
client.fine_tuning.jobs.retrieve("ftjob-3JBoNz2DNqXiaDBfSECqiyEN")


FineTuningJob(id='ftjob-3JBoNz2DNqXiaDBfSECqiyEN', created_at=1734408231, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-bQVIhPzVjnoakAjRyRFBoo5Y', result_files=[], seed=362833245, status='running', trained_tokens=None, training_file='file-H4CJZt8fPUph6ohyqsrAr1', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

- Use the chat completion model.

In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
model="ft:gpt-3.5-turbo-0125:personal::AfJG5wVQ",
messages=[
  {"role": "system", "content": "You are a helpful assistant."},
  {"role": "user", "content": "I have received damaged product"}
]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='We apologize for the inconvenience. Can you please provide your order number and a description of the damage so we can assist you further?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
